# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [16]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
indicator_list = pattern
indicator_list_higher = ['LinearReg']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15, 20, 25]},
                'LinearReg': {'timeperiod': [2, 3, 4, 5, 6, 8], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 72


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [02:32<00:00,  2.12s/it]


# Check local statistics

In [17]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_rank', ascending=False).head(20)
stat

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
66,STOCH_RSI_LinearReg,14,35,9,7,3,25,2,0,100,2.231608,-0.781250,123.160833,-78.125000,1.6145,1.9243,2.2736,2.5079,3.1990,2.9169,2.6705,2.7003,2.5584,2.4719,2.4203,2.3212,2.1791,2.1116,2.0650,2.0473,1.9837,1.9495,1.9515,1.9373,1.9563,1.9745,1.9767,1.8473,-0.30,-0.36,-0.60,-0.69,-0.96,-0.87,-0.73,-0.80,-0.70,-0.72,-0.66,-0.64,-0.76,-0.63,-0.66,-0.62,-0.71,-0.81,-0.94,-1.03,-1.13,-1.24,-1.15,-1.04
70,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,34,4.517596,-1.607083,119.598258,-54.640833,2.5622,3.0927,3.7470,4.1849,6.1106,5.1863,5.1057,5.3142,5.2999,5.2575,5.1253,5.0944,4.9967,4.6929,4.5337,4.3743,4.3001,4.2023,4.2380,4.1822,4.1856,4.2007,4.2054,4.2297,-0.30,-0.45,-0.75,-1.03,-1.54,-1.23,-1.85,-2.11,-2.19,-1.81,-1.68,-1.43,-1.31,-1.42,-1.44,-1.39,-1.22,-1.45,-1.96,-1.64,-2.04,-2.39,-2.74,-3.20
67,STOCH_RSI_LinearReg,14,35,9,7,3,25,3,0,55,3.127137,-1.241667,116.992562,-68.291667,1.8685,2.2621,2.7503,3.1126,4.4430,3.8766,3.7344,3.7945,3.4961,3.4284,3.3582,3.2554,3.2181,3.1029,3.0310,2.9902,2.9506,2.8876,2.9372,2.9264,2.9271,2.9673,2.9769,2.7559,-0.32,-0.37,-0.69,-0.97,-1.27,-1.11,-1.11,-1.22,-1.37,-1.16,-1.16,-1.00,-1.01,-0.83,-0.77,-0.94,-1.03,-1.22,-1.77,-1.65,-1.87,-2.14,-2.33,-2.49
69,STOCH_RSI_LinearReg,14,35,9,7,3,25,5,0,35,4.273646,-1.639167,114.577604,-57.370833,2.2849,2.9121,3.5680,3.9257,5.7619,4.8448,4.7111,4.8849,4.8594,4.7966,4.6568,4.6376,4.5717,4.3679,4.2388,4.1362,4.1135,4.0513,4.1161,4.1403,4.1690,4.2529,4.2608,4.3052,-0.27,-0.44,-0.84,-1.08,-1.47,-1.11,-1.68,-1.72,-1.84,-1.67,-1.47,-1.46,-1.25,-1.47,-1.59,-1.29,-1.41,-1.80,-2.08,-1.78,-2.29,-2.94,-2.99,-3.40
71,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,34,4.352196,-1.425000,113.974658,-48.450000,2.3509,2.8499,3.2018,3.7588,5.6254,4.8941,4.9455,5.1265,5.1491,5.0847,4.9956,4.9660,4.8904,4.6873,4.5752,4.3240,4.2376,4.2079,4.2019,4.1541,4.1439,4.1594,4.1767,3.7460,-0.30,-0.40,-0.62,-1.03,-1.33,-1.26,-1.49,-1.59,-1.91,-1.63,-1.42,-1.40,-1.14,-1.18,-1.44,-1.34,-1.10,-1.28,-1.82,-1.64,-1.83,-1.96,-2.32,-2.77
68,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,39,3.739613,-1.565417,106.844888,-61.051250,2.0860,2.6454,3.1643,3.5892,5.2198,4.3833,4.1824,4.2571,4.2343,4.1856,4.0278,3.9521,3.9086,3.7600,3.6703,3.6005,3.5857,3.5034,3.5567,3.5656,3.5925,3.6752,3.6830,3.7219,-0.33,-0.43,-0.84,-1.08,-1.61,-1.11,-1.68,-1.72,-1.67,-1.67,-1.43,-1.40,-1.20,-0.86,-1.25,-1.04,-1.04,-1.80,-2.08,-1.78,-2.29,-2.94,-2.99,-3.33
64,STOCH_RSI_LinearReg,14,35,9,7,3,20,6,0,30,4.233112,-1.505417,96.993375,-45.162500,2.8873,3.2508,4.0097,4.2848,5.3143,4.5402,4.4532,4.7014,4.7210,4.6804,4.6089,4.5789,4.5502,4.4311,4.3596,4.1793,4.0961,3.9872,4.0295,3.9688,3.9731,3.9923,3.9988,3.9978,-0.44,-0.48,-0.82,-1.03,-1.44,-1.15,-1.72,-1.87,-1.81,-1.81,-1.68,-1.43,-1.31,-1.42,-1.44,-1.39,-1.22,-1.45,-1.96,-1.64,-1.83,-1.96,-2.32,-2.51
60,STOCH_RSI_LinearReg,14,35,9,7,3,20,2,0,81,2.145138,-0.658750,92.756138,-53.358750,1.8001,2.0933,2.4689,2.6403,3.0444,2.7309,2.4634,2.5223,2.3992,2.3608,2.3076,2.2004,2.0418,1.9880,1.9479,1.9274,1.8695,1.8292,1.8283,1.8099,1.8257,1.8496,1.8457,1.6887,-0.34,-0.43,-0.62,-0.69,-0.82,-0.59,-0.57,-0.62,-0.60,-0.70,-0

# Research parameter influence

In [81]:
param = 'LinearReg_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,LinearReg_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'LinearReg_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            